In [1]:
import matplotlib.pyplot as plt
import alphashape
from descartes import PolygonPatch
from shapely.geometry import Point
from os import path
from KNN_filtration import *

### Function defns

In [13]:
# Takes in data matrix containing info for cell type to-be-curated and alphashape of structure in which 
# the cell type must be removed
# Returns filtered cell type data
def celltype_excluder(dat,alpha_shape):    
    points = np.array(dat[['x','y']])
    is_in_struct_arr = []
    for coord in points:
        is_in_struct = alpha_shape.intersects(Point(coord))
        is_in_struct_arr.append(is_in_struct)

    not_in_struct = [not i for i in is_in_struct_arr]
    result = dat[not_in_struct]
    
    return(result)

# Takes in data matrix containing info for cell type to-be-curated and alphashape of structure in which
# the cell must be maintained
# Returns filtered cell type data
def celltype_includer(dat,alpha_shape):
    points = np.array(dat[['x','y']])
    is_in_struct_arr = []
    for coord in points:
        is_in_struct = alpha_shape.intersects(Point(coord))
        is_in_struct_arr.append(is_in_struct)
    
    result = dat[is_in_struct_arr]
    
    return(result)

### Initialization

In [3]:
# cell type to-be-curated
cell_type = 'DCT'
# region of kidney 
section = 'cortex'

# input_path is path to file with all beads x features in an array, 
# features = {'barcode','x','y'}
input_path = 'coords.csv'
coords = pd.read_csv(input_path,index_col=0)
all_coords = np.array(coords)

# input_path is path to file with beads x features for marker beads in section of interest in an array
# features = {'barcode','x','y'}
input_path = 'markers.csv'
markers = pd.read_csv(input_path,index_col=0)

### Remove markers from relevant structures

In [5]:
# Glomeruli, granular cell clusters for CD-IC, CD-PC, DCT, PCT, and TAL
# MD for CD-IC, CD-PC, DCT, and PCT

if cell_type not in ['Podocyte','GC','MD']:
    has_glom = False
    has_gc = False
    has_MD = False
    alpha=0.01

    # input_path is path to data matrix with barcodes x features for all curated cell types in glomeruli
    # features = {'barcode','x','y'}
    input_path = 'glom_dat.csv'
    if path.exists(input_path):
        has_glom = True
        glom_dat = pd.read_csv(input_path,index_col=0)
        points = np.array(glom_dat[['x','y']])
        glom_alpha_shape = alphashape.alphashape(points,alpha=alpha)

    # input_path is path to data matrix with barcodes x features for all curated granular cell beads
    # features = {'barcode','x','y'}
    input_path = 'gc_dat.csv'
    if path.exists(input_path):
        has_gc = True
        gc_dat = pd.read_csv(input_path,index_col=0)
        points = np.array(gc_dat[['x','y']])
        gc_alpha_shape = alphashape.alphashape(points,alpha=alpha)

    # input_path is path to data matrix with barcodes x features for all curated MD beads
    # features = {'barcode','x','y'}
    input_path = 'MD_dat.csv'
    if path.exists(input_path):
        has_MD = True
        md_dat = pd.read_csv(input_path,index_col=0)
        points = np.array(md_dat[['x','y']])
        md_alpha_shape = alphashape.alphashape(points,alpha=alpha)

    if has_glom:
        markers = celltype_excluder(markers,glom_alpha_shape)
        if has_gc:
            markers = celltype_excluder(markers,gc_alpha_shape)
    if cell_type != 'TAL':
        if has_MD:
            markers = celltype_excluder(markers,md_alpha_shape)

### KNN filtration protocol

In [7]:
# specify parameters
# k = number of nearest neighbors to consider per marker bead
# threshold = number of nearest neighbors to maintain
k = 150
threshold = 5

In [8]:
eroded = get_markers_in_struct(all_coords, np.array(markers[['barcode','x','y']]), k, threshold)
eroded = pd.DataFrame(eroded)
eroded = eroded.rename(columns={0:'barcode',1:'x',2:'y'})

### Visualize eroded vs. maintained points

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(markers['x'],markers['y'],s=3,c='b')
plt.scatter(eroded['x'],eroded['y'],s=3,c='r')
plt.xlim(0,6000)
plt.ylim(0,6000)
colors=['b','r']
texts=['cell type marker','cell type filter 1']
patches = [ plt.plot([],[], marker="o", ms=3, ls="", linewidth=0.6,color=colors[i], 
                label="{:s}".format(texts[i]) )[0]  for i in range(len(texts)) ]
plt.legend(handles=patches, bbox_to_anchor=(1, 0.5), 
        loc='center left', ncol=1,fontsize='medium')
plt.show()

### Define alphashape of celltype

In [10]:
coords = np.array(eroded[['x','y']])
alpha = 0.015
celltype_alpha_shape = alphashape.alphashape(coords, alpha)

shape_list = []
if cell_type != 'TAL':
    if has_MD and has_gc and has_glom:
        shape_list = [glom_alpha_shape,gc_alpha_shape,MD_alpha_shape]
    elif has_glom and not has_MD and has_gc:
        shape_list = [glom_alpha_shape,gc_alpha_shape]
    elif has_glom and has_MD and not has_gc:
        shape_list = [glom_alpha_shape,MD_alpha_shape]
    elif has_glom and not has_MD and not has_gc:
        shape_list = [glom_alpha_shape]
else:
    if has_gc and has_glom:
        shape_list = [glom_alpha_shape,gc_alpha_shape]
    elif glom and not has_gc:
        shape_list = [glom_alpha_shape]

if len(shape_list)!=0:
    for shape in shape_list:
        celltype_alpha_shape = celltype_alpha_shape.difference(shape)

### Visualize alphashape

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.add_patch(PolygonPatch(celltype_alpha_shape, fc=(0,0,0,0),ec=(0,0,0,1)))
ax.scatter(*zip(*coords),s=3,c='r')
ax.set_xlim([0,6000])
ax.set_ylim([0,6000])

### Refine celltype structure by excluding points outside of alphashape

In [14]:
eroded = celltype_includer(eroded,celltype_alpha_shape)

### Visualize final product

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(markers['x'],markers['y'],s=3,c='b')
plt.scatter(eroded['x'],eroded['y'],s=3,c='r')
plt.xlim(0,6000)
plt.ylim(0,6000)
colors=['b','r']
texts=['cell type marker','cell type 2']
patches = [ plt.plot([],[], marker="o", ms=3, ls="", linewidth=0.6,color=colors[i], 
                label="{:s}".format(texts[i]) )[0]  for i in range(len(texts)) ]
plt.legend(handles=patches, bbox_to_anchor=(1, 0.5), 
        loc='center left', ncol=1,fontsize='medium')
plt.show()

### File output

In [ ]:
# out_path is path to output file
out_path = 'eroded.csv'
eroded.to_csv(out_path)